Living and Bubble are very small ---> data augmentation

How? ----> rotation, contrast, brighthness, ecc

In [1]:
import cv2
import os
import random
from albumentations import Compose, HorizontalFlip, VerticalFlip, ShiftScaleRotate, RandomBrightnessContrast

/Users/valentinaolaritei/Desktop/3 SEMESTER/.venv/lib/python3.12/site-packages/albumentations/check_version.py:51: UserWarning: Error fetching version info <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
  data = fetch_version_info()


In [ ]:
# Define different augmentation pipelines
augmentation_type_1 = Compose([
    HorizontalFlip(p=0.5),
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7),
    RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5)
])

augmentation_type_2 = Compose([
    VerticalFlip(p=0.5),
    ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=45, p=0.7),
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, p=0.5)
])

augmentation_type_3 = Compose([
    HorizontalFlip(p=0.7),
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=60, p=0.5),
    RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.2, p=0.7)
])

In [ ]:
# List of augmentations to randomly select from
augmentations = [augmentation_type_1, augmentation_type_2, augmentation_type_3]

def augment_image(image_path, label_path, output_dir, num_augments=3, times_per_type=1):
    # Load image
    image = cv2.imread(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Load label content if it exists
    if os.path.exists(label_path):
        with open(label_path, 'r') as file:
            label_content = file.read()

    # Apply multiple augmentations
    for augment_idx in range(num_augments):
        # Randomly choose an augmentation type
        augmentation = random.choice(augmentations)
        
        for repeat_idx in range(times_per_type):
            augmented = augmentation(image=image)['image']
            
            # Save the augmented image
            aug_image_name = f"{base_name}_aug_{augment_idx}_{repeat_idx}.png"
            cv2.imwrite(os.path.join(output_dir, aug_image_name), augmented)
            
            # Save the corresponding annotation
            aug_label_name = f"{base_name}_aug_{augment_idx}_{repeat_idx}.txt"
            if os.path.exists(label_path):  # Only save label if it exists
                with open(os.path.join(output_dir, aug_label_name), 'w') as file:
                    file.write(label_content)

In [ ]:
# Directories
input_dir = 'extracted_shapes'
output_dir = 'augmented_shapes'
os.makedirs(output_dir, exist_ok=True)

# Apply augmentation only to "Living" and "Bubble" images
for filename in os.listdir(input_dir):
    # Explicitly exclude any files containing "Non-Living" in the filename
    if filename.endswith('.png') and ("Living" in filename or "Bubble" in filename) and "Non-Living" not in filename:
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(input_dir, os.path.splitext(filename)[0] + '.txt')
        augment_image(image_path, label_path, output_dir)
